## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv ("C:/Users/betti/Desktop/World_weather_analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Todos Santos,MX,23.4500,-110.2167,78.01,84,19,6.31,few clouds
1,1,Marawi,PH,7.9986,124.2928,74.64,88,100,4.70,light rain
2,2,Berdigestyakh,RU,62.0989,126.6992,35.08,86,100,11.77,overcast clouds
3,3,Bluff,NZ,-46.6000,168.3333,54.50,77,98,13.11,overcast clouds
4,4,Ulladulla,AU,-35.3500,150.4667,82.76,58,51,9.60,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Todos Santos,MX,23.4500,-110.2167,78.01,84,19,6.31,few clouds
4,4,Ulladulla,AU,-35.3500,150.4667,82.76,58,51,9.60,broken clouds
5,5,Casa Nova,BR,-9.4167,-41.1333,76.98,66,97,10.51,overcast clouds
6,6,Luanda,AO,-8.8368,13.2343,76.62,80,93,6.73,overcast clouds
12,12,Dingle,PH,10.9995,122.6711,82.06,83,72,9.78,broken clouds
14,14,Tautira,PF,-17.7333,-149.1500,75.22,91,90,29.86,moderate rain
16,16,Nelson Bay,AU,-32.7167,152.1500,85.28,78,7,17.65,light rain
19,19,Pitimbu,BR,-7.4706,-34.8086,75.79,79,28,9.26,scattered clouds
23,23,Boa Vista,BR,2.8197,-60.6733,76.98,83,75,4.61,broken clouds
31,31,Sinnamary,GF,5.3833,-52.9500,77.54,89,94,6.20,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()
len(preferred_cities_df)

205

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Todos Santos,MX,78.01,few clouds,23.4500,-110.2167,
4,Ulladulla,AU,82.76,broken clouds,-35.3500,150.4667,
5,Casa Nova,BR,76.98,overcast clouds,-9.4167,-41.1333,
6,Luanda,AO,76.62,overcast clouds,-8.8368,13.2343,
12,Dingle,PH,82.06,broken clouds,10.9995,122.6711,
14,Tautira,PF,75.22,moderate rain,-17.7333,-149.1500,
16,Nelson Bay,AU,85.28,light rain,-32.7167,152.1500,
19,Pitimbu,BR,75.79,scattered clouds,-7.4706,-34.8086,
23,Boa Vista,BR,76.98,broken clouds,2.8197,-60.6733,
31,Sinnamary,GF,77.54,overcast clouds,5.3833,-52.9500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace(r'^\s*$', np.NaN, regex=True)
hotel_df = hotel_df.dropna()
hotel_df
hotel_df.count()

City                   190
Country                190
Max Temp               190
Current Description    190
Lat                    190
Lng                    190
Hotel Name             190
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Users/betti/Desktop/World_weather_analysis/Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} at {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))